In [1]:
import math
from gmpy2 import mpz
from time import time
import gmpy2

def pi_chudnovsky_bs(digits):
    """
    Compute int(pi * 10**digits)

    This is done using Chudnovsky's series with binary splitting
    """
    C = 640320
    C3_OVER_24 = C**3 // 24
    def bs(a, b):
        """
        Computes the terms for binary splitting the Chudnovsky infinite series

        a(a) = +/- (13591409 + 545140134*a)
        p(a) = (6*a-5)*(2*a-1)*(6*a-1)
        b(a) = 1
        q(a) = a*a*a*C3_OVER_24

        returns P(a,b), Q(a,b) and T(a,b)
        """
        if b - a == 1:
            # Directly compute P(a,a+1), Q(a,a+1) and T(a,a+1)
            if a == 0:
                Pab = Qab = mpz(1)
            else:
                Pab = mpz((6*a-5)*(2*a-1)*(6*a-1))
                Qab = mpz(a*a*a*C3_OVER_24)
            Tab = Pab * (13591409 + 545140134*a) # a(a) * p(a)
            if a & 1:
                Tab = -Tab
        else:
            # Recursively compute P(a,b), Q(a,b) and T(a,b)
            # m is the midpoint of a and b
            m = (a + b) // 2
            # Recursively calculate P(a,m), Q(a,m) and T(a,m)
            Pam, Qam, Tam = bs(a, m)
            # Recursively calculate P(m,b), Q(m,b) and T(m,b)
            Pmb, Qmb, Tmb = bs(m, b)
            # Now combine
            Pab = Pam * Pmb
            Qab = Qam * Qmb
            Tab = Qmb * Tam + Pam * Tmb
        return Pab, Qab, Tab
    # how many terms to compute
    DIGITS_PER_TERM = math.log10(C3_OVER_24/6/2/6)
    N = int(digits/DIGITS_PER_TERM + 1)
    # Calclate P(0,N) and Q(0,N)
    P, Q, T = bs(0, N)
    one_squared = mpz(10)**(2*digits)
    sqrtC = gmpy2.isqrt(10005*one_squared)
    return (Q*426880*sqrtC) // T

# The last 5 digits or pi for various numbers of digits
check_digits = {
        100 : 70679,
       1000 :  1989,
      10000 : 75678,
     100000 : 24646,
    1000000 : 58151,
   10000000 : 55897,
}

if __name__ == "__main__":
    #raise SystemExit
    for log10_digits in range(1,8): # 10자리 ~ 1000만자리
        digits = 10**log10_digits
        start =time()
        pi = pi_chudnovsky_bs(digits)
        f = open('d_' + str(digits), 'w')
        f.write(str(pi))
        f.close()
        print("chudnovsky_gmpy_mpz_bs: digits",digits,"time",time()-start)
        if digits in check_digits:
            last_five_digits = pi % 100000
            if check_digits[digits] == last_five_digits:
                print("Last 5 digits %05d OK" % last_five_digits)
            else:
                print("Last 5 digits %05d wrong should be %05d" % (last_five_digits, check_digits[digits]))

chudnovsky_gmpy_mpz_bs: digits 10 time 0.0009992122650146484
chudnovsky_gmpy_mpz_bs: digits 100 time 0.0009989738464355469
Last 5 digits 70679 OK
chudnovsky_gmpy_mpz_bs: digits 1000 time 0.0010564327239990234
Last 5 digits 01989 OK
chudnovsky_gmpy_mpz_bs: digits 10000 time 0.0023200511932373047
Last 5 digits 75678 OK
chudnovsky_gmpy_mpz_bs: digits 100000 time 0.03667879104614258
Last 5 digits 24646 OK
chudnovsky_gmpy_mpz_bs: digits 1000000 time 0.6439166069030762
Last 5 digits 58151 OK
chudnovsky_gmpy_mpz_bs: digits 10000000 time 12.621483564376831
Last 5 digits 55897 OK


In [2]:
'''
뉴턴 방법을 사용하면 계산 속도가 너무 느려서 gmpy2 모듈을 사용
https://www.craig-wood.com/nick/articles/pi-chudnovsky/pi_chudnovsky_bs_gmpy.py 에서 발췌

<첫 번째 수정>
Line 63 ---> sqrtC = (10005*one_squared).sqrt()
AttributeError: 'mpz' object has no attribute 'sqrt'
왜인지는 모르겠으나 발췌한 코드를 수정없이 사용하면 sqrt method가 존재하지 않아 오류가 발생함.

첫 번째 오류를 해결하기 위해 코드를 아래와 같이 수정함

추가함 : import gmpy2
수정함 : (10005*one_squared).sqrt() => gmpy2.sqrt(10005*one_squared)

<두 번째 수정>
수정한 코드를 실행하여보니if check_digits[digits] == last_five_digits
를 사용하는 과정에서 소숫점이 일치하지 않는 상황이 발생함
gmpy2.sqrt를 사용하지 않고 gmpy2.isqrt를 사용하였더니 정상적으로 작동됨을 확인하였다.
'''

"\n뉴턴 방법을 사용하면 계산 속도가 너무 느려서 gmpy2 모듈을 사용\nhttps://www.craig-wood.com/nick/articles/pi-chudnovsky/pi_chudnovsky_bs_gmpy.py 에서 발췌\n\n<첫 번째 수정>\nLine 63 ---> sqrtC = (10005*one_squared).sqrt()\nAttributeError: 'mpz' object has no attribute 'sqrt'\n왜인지는 모르겠으나 발췌한 코드를 수정없이 사용하면 sqrt method가 존재하지 않아 오류가 발생함.\n\n첫 번째 오류를 해결하기 위해 코드를 아래와 같이 수정함\n\n추가함 : import gmpy2\n수정함 : (10005*one_squared).sqrt() => gmpy2.sqrt(10005*one_squared)\n\n<두 번째 수정>\n수정한 코드를 실행하여보니if check_digits[digits] == last_five_digits\n를 사용하는 과정에서 소숫점이 일치하지 않는 상황이 발생함\ngmpy2.sqrt를 사용하지 않고 gmpy2.isqrt를 사용하였더니 정상적으로 작동됨을 확인하였다.\n"